In [13]:
import pandas as pd
import sqlite3

# Player Data

In [75]:
df1 = pd.read_csv('player_defense.csv')
df2 = pd.read_csv('player_gsc.csv')
df3 = pd.read_csv('player_misc.csv')
df4 = pd.read_csv('player_passing.csv')
df5 = pd.read_csv('player_possesion.csv')
df6 = pd.read_csv('player_shooting.csv')
df7 = pd.read_csv('player_time.csv')

In [76]:
players = df1[['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s']]
#create ID column
players.reset_index(inplace = True)
players['index'] = list(map(lambda x: x+1, players['index']))
players.rename(columns = {'index':'Id'}, inplace = True)
players['Nation'] = list(map(lambda x: x[-3:], players['Nation']))
players['Age'] = list(map(lambda x: x[:2], players['Age']))

defense = df1.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Tkl%', '%', 'Tkl+Int'], 1)
gsc = df2.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'SCA', 'SCA90', 'GCA', 'GCA90'], 1)
misc = df3.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Int', 'TklW'],1)
passing = df4.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Cmp.1', 'Att.1', 'Cmp%.1', 'Cmp%.2', 'Cmp%.3', 'A-xA', 'KP', '1/3'], 1)
possession = df5.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Touches', 'Live', 'Succ%', '1/3'], 1)
shooting = df6.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG'], 1)
playtime = df7.drop(['Rk', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s', 'Matches', '-9999', 'Mn/MP', 'Min%', '+/-', '+/-90', 'xG+/-', 'xG+/-90', 'On-Off.1'], 1)

C:\Users\Brian\AppData\Local\Temp/ipykernel_24328/3557681887.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['index'] = list(map(lambda x: x+1, players['index']))
C:\Users\Brian\AppData\Local\Temp/ipykernel_24328/3557681887.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['Nation'] = list(map(lambda x: x[-3:], players['Nation']))
C:\Users\Brian\AppData\Local\Temp/ipykernel_24328/3557681887.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [77]:
#add player id to each df
ids = players[['Id', 'Player']]
defense = pd.merge(ids, defense, how = 'inner', on='Player').drop(['Player'], 1)
gsc = pd.merge(ids, gsc, how = 'inner', on='Player').drop(['Player'], 1)
misc = pd.merge(ids, misc, how = 'inner', on='Player').drop(['Player'], 1)
passing = pd.merge(ids, passing, how = 'inner', on='Player').drop(['Player'], 1)
possession = pd.merge(ids, possession, how = 'inner', on='Player').drop(['Player'], 1)
shooting = pd.merge(ids, shooting, how = 'inner', on='Player').drop(['Player'], 1)
playtime = pd.merge(ids, playtime, how = 'inner', on='Player').drop(['Player'], 1)


C:\Users\Brian\AppData\Local\Temp/ipykernel_24328/544113327.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  defense = pd.merge(ids, defense, how = 'inner', on='Player').drop(['Player'], 1)
C:\Users\Brian\AppData\Local\Temp/ipykernel_24328/544113327.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  gsc = pd.merge(ids, gsc, how = 'inner', on='Player').drop(['Player'], 1)
C:\Users\Brian\AppData\Local\Temp/ipykernel_24328/544113327.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  misc = pd.merge(ids, misc, how = 'inner', on='Player').drop(['Player'], 1)
C:\Users\Brian\AppData\Local\Temp/ipykernel_24328/544113327.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels

,Id,Tkl,TklW,Def 3rd,Mid 3rd,Att 3rd,Tklvsdribble,Att,Tkl%,Past,...,Mid 3rd.1,Att 3rd.1,Blocks,Sh,ShSv,Pass,Int,Tkl+Int,Clr,Err
0,1,7.0,5,3.0,3.0,1.0,3.0,5.0,60.0,2.0,...,38.0,31.0,2.0,0.0,0.0,2.0,2,9.0,0.0,0.0
1,2,21.0,14,13.0,8.0,0.0,6.0,11.0,54.5,5.0,...,58.0,5.0,34.0,16.0,2.0,18.0,32,53.0,60.0,0.0
2,3,40.0,21,20.0,11.0,9.0,24.0,40.0,60.0,16.0,...,102.0,36.0,22.0,0.0,0.0,22.0,37,77.0,13.0,0.0
3,4,5.0,4,1.0,4.0,0.0,2.0,5.0,40.0,3.0,...,46.0,25.0,3.0,0.0,0.0,3.0,7,12.0,2.0,0.0
4,5,18.0,14,9.0,9.0,0.0,3.0,10.0,30.0,7.0,...,73.0,11.0,5.0,3.0,0.0,2.0,10,28.0,14.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,685,2.0,2,2.0,0.0,0.0,0.0,1.0,0.0,1.0,...,13.0,7.0,0.0,0.0,0.0,0.0,1,3.0,0.0,0.0
693,686,1.0,0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,...,6.0,14.0,1.0,0.0,0.0,1.0,0,1.0,0.0,0.0
694,687,7.0,3,7.0,0.0,0.0,1.0,5.0,20.0,4.0,...,19.0,3.0,6.0,2.0,0.0,4.0,16,23.0,38.0,0.0
695,688,13.0,7,8.0,3.0,2.0,8.0,15.0,53.3,7.0,...,48.0,16.0,23.0,3.0,0.0,20.0,28,41.0,24.0,0.0


In [79]:
#connect to mlsdb and create/update players table
with sqlite3.connect('mlsdb.sqlite') as conn:
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS players (id INT, name TEXT, nation TEXT, position TEXT, squad TEXT, age INT, born INT, games REAL)')
    conn.commit()
    players.to_sql('players', conn, if_exists='replace', index = False)
    
    c.execute('CREATE TABLE IF NOT EXISTS playerDefense (playerId INT, tkl REAL, tklW INT, tklDefThird REAL, tklMidThird REAL, tklAttThird REAL, tklVsDrib INT, tklAttVsDrib INT, past INT, press INT, pressW INT, pressDefThird INT, pressMidThird INT, pressAttThird INT, blk INT, blkSh INT, blkSot INT, blkPass INT, int INT, clr INT, err INT)')     
    conn.commit()
    defense.to_sql('stats', conn, if_exists='replace', index = False) 
    
    #check to make sure column names aren't duplicated in the browser
    c.execute('CREATE TABLE IF NOT EXISTS playerGsc(scaPassLive INT, scaPassDead INT, scaDrib INT, scaSh INT, scaFld INT, scaDef INT, gcaPassLive INT, gcaPassDead INT, gcaDrib INT, gcaSh INT, gcaFld INT, gcaDef INT)')
    conn.commit()
    gsc.to_sql('playerGsc', conn, if_exists='replace', index = False)
    
    c.execute('CREATE TABLE IF NOT EXISTS playerMisc(crdY INT, crdR INT, crdR2 INT, fls INT, fld INT, off Int, crs INT, pkWon INT, pkCon INT, og INT, recov INT, airWon INT, airLost INT)')
    conn.commit()
    misc.to_sql('playerMisc', conn, if_exists='replace', index = False)  
    
    c.execute('CREATE TABLE IF NOT EXISTS playerPassing(totDist INT, prgDist INT, shortCmp INT, shortAtt INT, medCmp INT, medAtt INT, lngCmp INT, lngAtt INT, ast INT, xA REAL, passPpa INT, crsPpa INT, progPasses INT)')
    conn.commit()
    passing.to_sql('playerPassing', conn, if_exists='replace', index = False)
    
    c.execute('CREATE TABLE IF NOT EXISTS playerPossession( defPen INT, defThird INT, midThird INT, attThird INT, attPen INT, succDrib INT, attDrib INT, playersBeat INT, megs INT, carries INT, totDist INT, prgDist INT, mishandles INT, tackled INT, passTarg INT, passRec INT, progPassesRec INT)')
    conn.commit()
    possession.to_sql('playerPossession', conn, if_exists='replace', index = False)
    
    c.execute('CREATE TABLE IF NOT EXISTS playerShooting(gls INT, sh INT, sot INT, avgDist REAL, fk INT, succPk INT, pkAtt INT, xG REAL, npxG REAL)')
    conn.commit()
    shooting.to_sql('playerShooting', conn, if_exists='replace', index = False)    
    
    c.execute('CREATE TABLE IF NOT EXISTS playerTime(min INT, starts INT, minPerStart INT, complete INT, subbed INT, minPerSub INT, unSub INT, ppm REAL, onG INT, onGA INT, onOff REAL, onxG REAL, onxGa REAL)')
    conn.commit()
    playtime.to_sql('playerTime', conn, if_exists='replace', index = False) 

# Squad Data

In [184]:
eastdf = pd.read_csv('squad/eastern_conf.csv')
westdf = pd.read_csv('squad/western_conf.csv')
df1 = pd.concat([eastdf, westdf])
df2 = pd.read_csv('squad/passing.csv')
df3 = pd.read_csv('squad/gsc.csv')
df4 = pd.read_csv('squad/shooting.csv')
df5 = pd.read_csv('squad/defense.csv')
df6= pd.read_csv('squad/posession.csv')
df7= pd.read_csv('squad/goalkeeping.csv')
df8= pd.read_csv('squad/squad_advanced_goalkeeping.csv')
df9 = pd.read_csv('squad/misc.csv')

In [185]:
df1.drop(['Rk', 'MP', 'GD', 'Pts/MP', 'xGD/90', 'Top Team Scorer', 'Goalkeeper', 'Notes'], 1, inplace = True)
df2.drop(['90s', 'Cmp%', 'Cmp%.1', 'Cmp%.2', 'Cmp%.3', 'A-xA', 'KP'], 1, inplace = True)
df3.drop(['# Pl', '90s', 'SCA90', 'GCA90'], 1, inplace = True)
df4.drop(['# Pl', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG'], 1, inplace = True)
df5.drop(['# Pl', '90s', 'Tkl%', '%', 'Tkl+Int'], 1, inplace = True)
df6.drop(['# Pl', '90s', 'Touches', 'Succ%', 'Rec%'], 1, inplace = True)
df7.drop(['# Pl', 'MP', 'Starts', 'Min', '90s', 'GA90', 'Save%', 'W', 'D', 'L', 'CS%', 'Save%', 'PKatt', 'Save%'], 1, inplace = True)
df8.drop(['# Pl', '90s', 'PSxG/SoT', 'Cmp%', 'Stp%','#OPA/90', 'GA', 'PKA'], 1, inplace = True)
df9.drop(['# Pl', '90s', 'Won%'], 1, inplace = True)


C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/1833448445.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df1.drop(['Rk', 'MP', 'GD', 'Pts/MP', 'xGD/90', 'Top Team Scorer', 'Goalkeeper', 'Notes'], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/1833448445.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df2.drop(['90s', 'Cmp%', 'Cmp%.1', 'Cmp%.2', 'Cmp%.3', 'A-xA', 'KP'], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/1833448445.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df3.drop(['# Pl', '90s', 'SCA90', 'GCA90'], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/1833448445.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.dro

In [186]:
df = pd.merge(df1, df2, how = 'inner', on='Squad')
df = pd.merge(df, df3, how = 'inner', on='Squad')
df = pd.merge(df, df4, how = 'inner', on='Squad')
df = pd.merge(df, df5, how = 'inner', on='Squad')
df = pd.merge(df, df6, how = 'inner', on='Squad')
df = pd.merge(df, df7, how = 'inner', on='Squad')
df = pd.merge(df, df8, how = 'inner', on='Squad')
df = pd.merge(df, df9, how = 'inner', on='Squad')
df.reset_index(inplace = True)
df['index'] = list(map(lambda x: x+1, df['index']))

C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/291323891.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Att_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, df8, how = 'inner', on='Squad')


In [187]:
squads = df[['index', 'Squad', 'W', 'D', 'L', 'Last 5', 'Attendance']]
squads.rename(columns = {'index':'id'}, inplace = True)
squad_stats = df.drop(['Squad', 'W', 'D', 'L', 'Last 5', 'Attendance', '# Pl'], 1)
squad_stats.rename(columns = {'index':'squad_id'}, inplace = True)

#make dict of squads and ids
squad_dict = dict(zip(squads['Squad'], squads['id']))


C:\Users\Brian\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/941489506.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  squad_stats = df.drop(['Squad', 'W', 'D', 'L', 'Last 5', 'Attendance', '# Pl'], 1)


In [188]:
#load squad against stats into df. 
df1 = pd.read_csv('squad/passing_against.csv')
df2 = pd.read_csv('squad/gsc_against.csv')
df3 = pd.read_csv('squad/shooting_against.csv')
df4 = pd.read_csv('squad/defense_against.csv')
df5= pd.read_csv('squad/posession_against.csv')
df6= pd.read_csv('squad/goalkeeping_against.csv')
df7 = pd.read_csv('squad/misc_against.csv')


In [189]:
df1.drop(['90s', 'Cmp%', 'Cmp%.1', 'Cmp%.2', 'Cmp%.3', 'A-xA', 'KP'], 1, inplace = True)
df2.drop(['# Pl', '90s', 'SCA90', 'GCA90'], 1, inplace = True)
df3.drop(['# Pl', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG'], 1, inplace = True)
df4.drop(['# Pl', '90s', 'Tkl%', '%', 'Tkl+Int'], 1, inplace = True)
df5.drop(['# Pl', '90s', 'Touches', 'Succ%', 'Rec%'], 1, inplace = True)
df6.drop(['# Pl', 'MP', 'Starts', 'Min', '90s', 'GA90', 'Save%', 'W', 'D', 'L', 'CS%', 'Save%'], 1, inplace = True)
df7.drop(['# Pl', '90s', 'Won%'], 1, inplace = True)

C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/2553514346.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df1.drop(['90s', 'Cmp%', 'Cmp%.1', 'Cmp%.2', 'Cmp%.3', 'A-xA', 'KP'], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/2553514346.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df2.drop(['# Pl', '90s', 'SCA90', 'GCA90'], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/2553514346.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df3.drop(['# Pl', '90s', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'npxG/Sh', 'G-xG', 'np:G-xG'], 1, inplace = True)
C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/2553514346.py:4: FutureWarning: In a future version of pandas all arguments of Data

In [190]:
df = pd.merge(df1, df2, how = 'inner', on='Squad')
df = pd.merge(df, df3, how = 'inner', on='Squad')
df = pd.merge(df, df4, how = 'inner', on='Squad')
df = pd.merge(df, df5, how = 'inner', on='Squad')
df = pd.merge(df, df6, how = 'inner', on='Squad')
squad_against = pd.merge(df, df7, how = 'inner', on='Squad')
squad_against['Squad_id'] = list(map(lambda x: squad_dict[x[3:]], squad_against['Squad']))
squad_against.drop('Squad', 1, inplace = True)

C:\Users\Brian\AppData\Local\Temp/ipykernel_1168/4039849574.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  squad_against.drop('Squad', 1, inplace = True)


In [191]:
  #normalize databases further, one table is going to be too confusing

C:\Users\Brian\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


OperationalError: near ")": syntax error

str

,squad_id,GF,GA_x,Pts,xG_x,xGA,xGD,Cmp_x,Att_x,TotDist_x,...,Off,Crs,Int_y,TklW_y,PKwon,PKcon,OG_y,Recov,Won,Lost
0,1,28,20,29,28.7,24.0,4.7,4767,6857,96341,...,28,225,319,265,3,1,0,1820,379,380
1,2,22,13,29,26.4,18.9,7.5,4397,6268,91783,...,26,179,280,207,1,3,0,1486,365,361
2,3,31,17,28,31.8,19.5,12.3,7420,9026,146285,...,27,225,262,185,2,0,1,1430,276,241
3,4,20,23,25,19.7,24.2,-4.5,6449,7917,123356,...,23,199,229,183,0,3,0,1196,232,217
4,5,27,30,24,27.3,25.7,1.6,5518,7224,109166,...,30,171,294,212,3,5,1,1403,263,255
5,6,27,26,24,22.8,20.2,2.6,7574,9192,155249,...,20,282,230,162,3,1,1,1356,258,228
6,7,17,25,21,19.0,23.7,-4.6,5588,6956,114180,...,22,152,281,192,2,3,0,1087,182,212
7,8,20,18,21,21.9,19.5,2.5,6421,7953,127560,...,28,201,290,171,0,1,1,1331,224,287
8,9,17,22,20,19.8,22.2,-2.3,7370,9166,149843,...,20,219,282,178,1,3,0,1423,248,276
9,10,24,24,19,24.4,22.4,2.0,7387,8817,147222,...,32,219,260,168,1,2,0,1261,214,200


In [169]:
pd.concat([eastdf, westdf])

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,CF MontrÃ©al,17,9,2,6,32,29,3,29,1.71,23.8,18.4,5.4,0.32,L W L W W,13904,Djordje Mihailovic - 7,Sebastian Breza,NaN
1,2,NY Red Bulls,18,8,5,5,28,20,8,29,1.61,28.7,24.0,4.7,0.26,W L W L W,15296,Lewis Morgan - 8,Carlos Coronel,NaN
2,3,Philadelphia,17,7,8,2,22,13,9,29,1.71,26.4,18.9,7.5,0.44,W D D W L,17621,DÃ¡niel Gazdag - 7,Andre Blake,NaN
3,4,NYCFC,16,8,4,4,31,17,14,28,1.75,31.8,19.5,12.3,0.77,W W D L D,17653,ValentÃ­n Castellanos - 8,Sean Johnson,NaN
4,5,Orlando City,17,7,4,6,20,23,-3,25,1.47,19.7,24.2,-4.5,-0.27,D L D W L,17353,Ercan Kara - 6,Pedro Gallese,NaN
5,6,FC Cincinnati,17,7,3,7,27,30,-3,24,1.41,27.3,25.7,1.6,0.10,L L D W D,21343,Brandon Vazquez - 8,Roman Celentano,NaN
6,7,New England,17,6,6,5,27,26,1,24,1.41,22.8,20.2,2.6,0.15,D W D W D,18413,Adam Buksa - 7,Matt Turner,NaN
7,8,Inter Miami,16,6,3,7,17,25,-8,21,1.31,19.0,23.7,-4.6,-0.29,D W W L W,12562,Leonardo Campana - 7,Drake Callender,NaN
8,9,Columbus Crew,16,5,6,5,20,18,2,21,1.31,21.9,19.5,2.5,0.16,L W D D W,18993,Lucas ZelarayÃ¡n - 4,Eloy Room,NaN
9,10,Charlotte FC,18,6,2,10,17,22,-5,20,1.11,19.8,22.2,-2.3,-0.13,L W D L L,36404,Karol Åšwiderski - 4,Kristijan Kahlina,NaN
